In [ ]:
The goal of this assignment is to implement A* search that will find two different factory sequences:

    1. find the factory sequence with the smallest number of stops (the smallest number of factories visited).
    2. find the factory sequence with the smallest number of miles traveled.

Solve this problem using A* search. Your report should specify your state representation, actions, and your heuristic. 
The heuristic need not be very smart, but it needs to be non-zero, and it needs to be admissible and consistent.

In [95]:
# W0, W1, W2, W3, W4
W =[ ['A', 'E', 'D', 'C', 'A'],
     ['B', 'E', 'A', 'C', 'D'],
     ['B', 'A', 'B', 'C', 'E'],
     ['D', 'A', 'D', 'B', 'D'],
     ['B', 'E', 'C', 'B', 'D'] ]

W2 =[ ['A', 'E', 'D', 'C', 'A','B'],
     ['B', 'E', 'A', 'C', 'D', 'C'],
     ['B', 'A', 'B', 'C', 'E', 'D'],
     ['D', 'A', 'D', 'B', 'D', 'A'],
     ['B', 'E', 'C', 'B', 'D', 'C'] ]

# pt_sample = [W#, # widget, (pt1, pt2, pt3, pt4, pt5), steps]
pt_sample = [0, 2, [2, 2, 2, 2, 2], 4]
# heuristic for smallest stops
def heuristic(pt, N, W):
    distance = [0, 0, 0, 0, 0]
    for i in range(5): # go over 5 widgets with length N
        curr = pt[2][i]
        temp = [0, 0, 0, 0, 0]
        if i == pt[0]:
            curr += 1 
        while curr != len(W[i]):
            if(W[i][curr] == 'A'):
                temp[0] += 1
            elif(W[i][curr] == 'B'):
                temp[1] += 1
            elif(W[i][curr] == 'C'):
                temp[2] += 1
            elif(W[i][curr] == 'D'):
                temp[3] += 1
            elif(W[i][curr] == 'E'):
                temp[4] += 1
            curr += 1 
        for a in range(len(temp)):
            if(distance[a] < temp[a]):
                distance[a] = temp[a]
    return sum(distance) + pt[-1] # H(curr) + G(curr)
print(heuristic(pt_sample, 6, W2))

11


In [96]:
# s = [0,[-1, -1, (0, 0, 0, 0, 0), step]]
import heapq
import copy

def A_star(N, W):
    curr_char = {}
    loc = [0, 0, 0, 0, 0]
    heap = []
    # initialize points
    for i in range(5):
        if curr_char.get(W[i][0]) == None:
            curr_char[W[i][0]] = 1
            temp_pt = [i, 0, loc, 1]
            temp_h = heuristic(temp_pt, N, W)
            heapq.heappush(heap, [temp_h, temp_pt,W[i][0]])
    # start search
    while heap:
        curr_char.clear() # clear the dictionary
        curr_pt = heapq.heappop(heap) 
        curr_loc = copy.deepcopy(curr_pt[1][2])
        curr_w = W[curr_pt[1][0]][curr_pt[1][1]]
        # update new location
        for i in range(5):
            if curr_loc[i] < len(W[i]):
                if W[i][curr_loc[i]] == curr_w:
                    curr_loc[i] += 1
        if sum(curr_loc) == 5 * N:
            print("final result:", curr_pt)
            break
        # push all new points
        for i in range(5):
            if curr_loc[i] < len(W[i]):
                if curr_char.get(W[i][curr_loc[i]]) == None:
                    curr_char[W[i][curr_loc[i]]] = 1
                    temp_pt = [i, curr_loc[i], curr_loc, curr_pt[1][3] + 1]# add one to the original step
                    temp_h = heuristic(temp_pt, N, W)
                    heapq.heappush(heap, [temp_h, temp_pt,curr_pt[2] + W[i][curr_loc[i]]])
        

In [98]:
A_star(5, W)

Total Points: 960
final result: [11, [2, 4, [5, 5, 4, 5, 5], 11], 'BDAEDCBACDE']


In [22]:
import heapq
import copy
#     A    B    C     D    E
D = [[0, 1064, 673, 1401, 277],
     [1064, 0, 958, 1934, 337],
     [673, 958, 0, 1001, 399],
     [1401, 1934, 1001, 0, 387],
     [277, 337, 399, 387, 0]]


# The credits for this code go to UIUC_CS440_FALL18_HW01 by Zhenwei Wu, Yuji Chai, Zhoushi Zhu 
# uptree
def addelement_dset(dset, num):
    for i in range(num):
        dset.append(-1)

def find_dset(dset, elem):
    if(dset[elem] < 0):
        return elem
    else:
        find_result = find_dset(dset,dset[elem])
        dset[elem] = find_result
        return find_result

def setunion_dset(dset, a, b):
    root1 = find_dset(dset,a)
    root2 = find_dset(dset,b)
    
    newsize = dset[root1] + dset[root2]
    if(dset[root1] >= dset[root2]):
        dset[root2] = root1
        dset[root1] = newsize
    else:
        dset[root1] = root2
        dset[root2] = newsize
# generate distance = [1401, 'D2', 'A4']       
def hForAll(ls):
    visit_dic = {}
    distance_list = []
    for i in range(len(ls)):
        for j in range(i+1, len(ls)):
            if ls[i][0] != ls[j][0]:
                temp_distance = D[ord(ls[i][0])-65][ord(ls[j][0])-65]
                distance_list.append([temp_distance, [ls[i][0], str(ls[i][1])],[ls[j][0], str(ls[j][1])]])
    return distance_list

# Krusal
def mst(ls):
    #print("ls:",ls)
    dset = []
    Edge = hForAll(ls)
    E_heap = []
    distance_dic = {}
    addelement_dset(dset, len(ls))
    ct = 0 # give each widge a special number, for uptree
    for elm in Edge:
        heapq.heappush(E_heap, elm)
        if distance_dic.get(elm[1][0] + elm[1][1]) == None:
            distance_dic[elm[1][0] + elm[1][1]] = ct
            ct += 1
        if distance_dic.get(elm[2][0] + elm[2][1]) == None:
            distance_dic[elm[2][0] + elm[2][1]] = ct
            ct += 1
    edge_count = 0
    total_dist = 0
    while E_heap:
        if edge_count == len(ls) - 1:
            break
        it = heapq.heappop(E_heap)
        A = distance_dic[it[1][0] + it[1][1]]
        B = distance_dic[it[2][0] + it[2][1]]
        #for i in Edge:
        if find_dset(dset,A) != find_dset(dset,B):
            #print("cur_edge:", it, " ", it[0])
            edge_count += 1
            total_dist += it[0]
            setunion_dset(dset,A, B)
        #print("total_dist:", total_dist)
    return total_dist
##########################################################################    
sample = [0, 2, [2, 3, 2, 2, 2], 2789]

def h_distance(pt, W):
    char_dic = {}
    w_ls = []
    for i in range(5):
        curr = pt[2][i]
        while curr != len(W[i]):
            if char_dic.get(W[i][curr] + str(curr)) == None:
                char_dic[W[i][curr] + str(curr)] = 1
                w_ls.append([W[i][curr],curr])
            curr += 1 
    return mst(w_ls) + pt[-1]

#print(h_distance(sample))
    

In [23]:
print(h_distance(sample, W2))

6712


In [24]:
# pt_sample = [W#, # widget, (pt1, pt2, pt3, pt4, pt5), steps]
# s = [0,[-1, -1, [0, 0, 0, 0, 0], step]]
import heapq
import copy

def A_star_D(N, W): # widget with length N; W list
    curr_char = {}
    loc = [0, 0, 0, 0, 0]
    heap = []
    # initialize points
    for i in range(5):
        if curr_char.get(W[i][0]) == None:
            curr_char[W[i][0]] = 1
            temp_pt = [i, 0, loc, 0]
            temp_h = h_distance(temp_pt, W)
            heapq.heappush(heap, [temp_h, temp_pt,W[i][0]])
    # start search
    while heap:
        curr_char.clear() # clear the dictionary
        curr_pt = heapq.heappop(heap)
        curr_loc = copy.deepcopy(curr_pt[1][2])
        curr_w = W[curr_pt[1][0]][curr_pt[1][1]]
        # update new location
        for i in range(5):
            if curr_loc[i] < len(W[i]):
                if W[i][curr_loc[i]] == curr_w:
                    curr_loc[i] += 1
        if sum(curr_loc) == 5 * N:
            print("Final: ",curr_pt)
            break
        # push all new points
        for i in range(5):
            possible_next = ""
            if curr_loc[i] < len(W[i]):
                if curr_char.get(W[i][curr_loc[i]]) == None:
                    curr_char[W[i][curr_loc[i]]] = 1
                    
                    temp_pt = [i, curr_loc[i], curr_loc, curr_pt[1][3] + 
                               D[ord(curr_w)-65][ord(W[i][curr_loc[i]])-65]]# add new distance to the original step
                    temp_h = h_distance(temp_pt, W)
                    #possible_next += str(temp_pt) + "-->" + W[i][curr_loc[i]] + "; "
                    heapq.heappush(heap, [temp_h, temp_pt,curr_pt[2] + W[i][curr_loc[i]]])
            
        

In [15]:
A_star(6, W2)

final result: [13, [0, 5, [5, 6, 6, 6, 6], 13], 'BDEABCDBEDCAB']


In [97]:
A_star(5, W)

Total Points: 960
final result: [11, [2, 4, [5, 5, 4, 5, 5], 11], 'BDAEDCBACDE']


In [26]:
A_star_D(6, W2)

Final:  [9671, [2, 5, [6, 6, 5, 6, 6], 9671], 'BEDADCBEDCABED']


In [27]:
A_star_D(5, W)

Final:  [7043, [1, 4, [5, 4, 5, 4, 4], 7043], 'BEDAEDCBCAED']


In [85]:
from random import shuffle
x = [[i] for i in range(10)]
shuffle(x)

def problem_generator(N):
    new_widget = []
    string = 'ABCDE'
    for i in range(5):
        last = 'L'
        temp = []
        for k in range(N):
            temp_rand = [0,1,2,3,4] 
            shuffle(temp_rand) # get true random number
            temp_char = string[temp_rand[0]]
            while temp_char == last:
                shuffle(temp_rand)
                temp_char = string[temp_rand[0]]
            temp.append(str(temp_char))
            last = temp_char
        new_widget.append(temp)
    return new_widget

In [91]:
W_test = problem_generator(4)
for i in W_test:
    print(i)

['C', 'D', 'A', 'E']
['E', 'C', 'B', 'D']
['B', 'A', 'C', 'B']
['A', 'B', 'A', 'B']
['E', 'C', 'B', 'A']


In [92]:
A_star(4, W_test)

final result: [9, [0, 3, [3, 4, 4, 4, 4], 9], 'BAECBDABE']


In [75]:
string = 'ABCD'
print(string[0])

A
